In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))
from src.interaction.message import ChatMessageGenerator
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_community.llms.bedrock import Bedrock
from langchain_aws import ChatBedrock
from langchain_community.embeddings import BedrockEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings
from src.credential_manager.LocalCredentials import LocalCredentials
from src.model.model_catalogue import ModelType, EmbeddingType, Providers, ModelCatalogue
from src.model.wrappers import ModelWrapper, EmbeddingWrapper
from src.data.attachments import AttachmentTypes, Attachment


c:\Users\Student\Documents\University\Fifth Year\COM4520\Psycore\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = ModelCatalogue._models["bakllava_7b"]
model_wrapper = ModelWrapper(model)
attach = [Attachment(AttachmentTypes.IMAGE, "dog.jpeg", True),Attachment(AttachmentTypes.IMAGE, "cat.jpeg", True)]
Attachment.extractAttachmentList(attach)
print(model.family)

fake_wrapper = ModelWrapper(ModelCatalogue._models["oai_4o_latest"])
generator = ChatMessageGenerator(fake_wrapper,"Describe all user prompts")
prepped_message = generator.prepResponse(attachments=attach)
print(prepped_message)

llava
('user', [{'type': 'text', 'text': '{prompt}'}, {'type': 'image_url', 'image_url': {'url': 'data:image/jpeg;base64,{attachment0}'}}, {'type': 'image_url', 'image_url': {'url': 'data:image/jpeg;base64,{attachment1}'}}])


In [5]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

generator.chat.append(prepped_message)
template = ChatPromptTemplate(generator.chat)
chain = template | model_wrapper.model
chain2 = template | fake_wrapper.model

input_variables = Attachment.attachmentListMapping(attach)
print(input_variables)
input_variables.update({"prompt": "What's in the images."})
response = chain.invoke(input_variables)
response2 = chain2.invoke(input_variables)
print(response)
print(response2)

{'attachment0': '...', 'attachment1': '...'}
content=' The image features a white kitten with blue eyes and light brown fur sitting on top of a leopard print fabric.' additional_kwargs={} response_metadata={'model': 'bakllava:7b', 'created_at': '2025-02-20T17:38:18.0944811Z', 'done': True, 'done_reason': 'stop', 'total_duration': 44048163500, 'load_duration': 28631601600, 'prompt_eval_count': 1178, 'prompt_eval_duration': 10270000000, 'eval_count': 26, 'eval_duration': 5080000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-f9839738-0f89-4c41-a462-0434a2a89b09-0' usage_metadata={'input_tokens': 1178, 'output_tokens': 26, 'total_tokens': 1204}
content='The first image features a golden retriever puppy sitting on a fabric background. The puppy has fluffy fur and is wearing a pink collar.\n\nThe second image shows a light-colored kitten with blue eyes. The kitten is resting on a leopard-print surface and looking upwards.' additional_kwargs={'ref